Jupyter notebook för att göra kluster för Livsmedelsverkets databas

In [73]:
import sqlite3
import numpy as np
from sklearn.cluster import KMeans
import plotly
import plotly.graph_objs as go
import plotly.plotly as py

In [135]:
# Variable to toggle printouts
debug = True

Vi har konverterat en nedladdad Excel-fil till csv och sen till Sqlite 3. 

Den innehåller alla Livsmedelsverkets 59 näringsämnen. Här jobbar vi bara med kolhydrater, fett och protein.

Först läser vi in från databasen och lägger värdena i arrayen *result*

In [136]:
conn = sqlite3.connect('livs.db')  # Create db and establish connection
conn.row_factory = sqlite3.Row
curs = conn.cursor()
columnObject = []

        #Kolla om klusterkolumnen redan finns, annars lägg till den        
for row in conn.execute('PRAGMA table_info (livs);'):
    columnObject.append(row)
#print (columnObject)
columns = np.array(columnObject).tolist()

conn.close()

In [137]:
for column in columns:
    print (column[0],column[1])

0 Livsmedelsnamn
1 Livsmedelsnummer
2 Energi_kcal
3 Energi_kJ
4 Kolhydrater_g
5 Fett_g
6 Protein_g
7 Fibrer_g
8 Vatten_g
9 Alkohol_g
10 Aska_g
11 Monosackarider_g
12 Disackarider_g
13 Sackaros_g
14 Fullkorn_totalt_g
15 Sockerarter_g
16 Summa_mattade_fettsyror_g
17 Fettsyra_4_0_10_0_g
18 Fettsyra_12_0_g
19 Fettsyra_14_0_g
20 Fettsyra_16_0_g
21 Fettsyra_18_0_g
22 Fettsyra_20_0_g
23 Summa_enkelomattade_fettsyror_g
24 Fettsyra_16_1_g
25 Fettsyra_18_1_g
26 Summa_fleromattade_fettsyror_g
27 Fettsyra_18_2_g
28 Fettsyra_18_3_g
29 Fettsyra_20_4_g
30 EPA_Fettsyra_20_5_g
31 DPA_Fettsyra_22_5_g
32 DHA_Fettsyra_22_6_g
33 Kolesterol_mg
34 Retinol_mikrog
35 Vitamin_A_mikrog
36 beta_Karoten_mikrog
37 Vitamin_D_mikrog
38 Vitamin_E_mg
39 Vitamin_K_mikrog
40 Tiamin_mg
41 Riboflavin_mg
42 Vitamin_C_mg
43 Niacin_mg
44 Niacinekvivalenter_mg
45 Vitamin_B6_mg
46 Vitamin_B12_mikrog
47 Folat_mikrog
48 Fosfor_mg
49 Jod_mikrog
50 Jarn_mg
51 Kalcium_mg
52 Kalium_mg
53 Magnesium_mg
54 Natrium_mg
55 Salt_g
56 Selen_

In [150]:
# Välj vilka näringsvärden som ska läsas ur databasen. Gör en lista av dem i valueList
valueList = range(2,7)
columnsToQuery=[]
for index, item in enumerate(valueList):
    columnsToQuery.append(columns[item][1])

# Gör en sträng av listan av kolumner
columnsToQuery = ', '.join(columnsToQuery)
if debug:
    print (columnsToQuery)

Energi_kcal, Energi_kJ, Kolhydrater_g, Fett_g, Protein_g


In [151]:
conn = sqlite3.connect('livs.db')  # Create db and establish connection
conn.row_factory = sqlite3.Row
curs = conn.cursor()
result = []
rows = curs.execute('select '+columnsToQuery+' from livs limit 300')
for row in rows:
        result.append(row)

rows = np.array(result)
print (rows)

conn.close()

[[  6.56300000e+02   2.74600000e+03   0.00000000e+00   7.10000000e+01
    7.00000000e+00]
 [  7.63000000e+02   3.19260000e+03   0.00000000e+00   8.50000000e+01
    2.80000000e+00]
 [  8.84300000e+02   3.70000000e+03   0.00000000e+00   1.00000000e+02
    0.00000000e+00]
 ..., 
 [  2.61000000e+01   1.09100000e+02   4.40000000e+00   1.00000000e-01
    1.00000000e+00]
 [  2.95000000e+01   1.23400000e+02   4.10000000e+00   1.50000000e-01
    1.56000000e+00]
 [  3.14000000e+01   1.31300000e+02   4.70000000e+00   3.00000000e-01
    1.13000000e+00]]


Livsmedelsnumret ska inte vara med i klustringen så vi gör två matriser. 

allFactors innehåller Livsmedelsnumret, och dataSet allt utom Livsmedelsnumret

In [152]:
allFactors = np.array(result)
dataSet = allFactors[:,1:]

För klustringsalgoritmerna behöver vi använda numpy och göra en np-array

För fem livsmedel, kolumner för kolhydrater, fett och protein

<pre>[[   0.    71.     7. ]
 [   0.    85.     2.8]
 [   0.   100.     0. ]
 [   0.   100.     0. ]
 [   0.5   80.     0.5]]</pre>



In [153]:
if debug:
    print (dataSet)

[[  2.74600000e+03   0.00000000e+00   7.10000000e+01   7.00000000e+00]
 [  3.19260000e+03   0.00000000e+00   8.50000000e+01   2.80000000e+00]
 [  3.70000000e+03   0.00000000e+00   1.00000000e+02   0.00000000e+00]
 ..., 
 [  1.09100000e+02   4.40000000e+00   1.00000000e-01   1.00000000e+00]
 [  1.23400000e+02   4.10000000e+00   1.50000000e-01   1.56000000e+00]
 [  1.31300000e+02   4.70000000e+00   3.00000000e-01   1.13000000e+00]]


Funktion för att göra själva klustringen enligt k-means.

K-means förklaras relativt enkelt här: http://bigdata-madesimple.com/possibly-the-simplest-way-to-explain-k-means-algorithm/

In [154]:
def cluster(dataSet):

    kmeans = KMeans(n_clusters=numClusters)
    kmeans.fit(dataSet)

    centroids = kmeans.cluster_centers_
    assignedCluster = kmeans.labels_

    if debug:
        print ("Centroids \n", centroids)
        print
        print (assignedCluster)

    return assignedCluster

När man kör klustringsalgoritmen returneras en array av vilket kluster varje livsmedel tillhör. 

Man kan även få ut mittpunkterna i varje kluster genom variabeln *centroids*

Ett exempel: 

<pre>
centroids:
[[   0.    100.      0.  ]
 [   0.25   82.5     1.65]
 [   0.     71.      7.  ]]

assignedCluster:
[2 1 0 0 1]</pre>

In [155]:
numClusters = 5
assignedCluster = cluster(dataSet)

Centroids 
 [[  1.56319062e+03   3.43326562e+01   2.03017187e+01   1.11120313e+01]
 [  3.09320714e+02   8.50814286e+00   2.44671429e+00   3.59042857e+00]
 [  3.62284167e+03   3.41666667e-02   9.77341667e+01   3.55000000e-01]
 [  1.03321212e+03   2.89581818e+01   9.87227273e+00   9.12409091e+00]
 [  2.69716667e+03   1.76555556e+00   7.16311111e+01   1.04500000e+00]]
[4 2 2 2 4 4 4 4 4 4 4 4 0 0 0 2 0 0 4 4 0 0 0 4 4 4 2 2 2 2 2 2 2 2 3 4 0
 3 0 3 3 1 4 0 0 0 1 1 1 0 4 0 4 3 3 0 0 0 3 3 1 1 1 1 3 1 1 3 0 0 0 3 0 1
 1 3 3 1 3 0 3 3 0 0 3 3 0 0 0 3 3 0 0 0 0 3 3 3 3 1 0 0 1 1 1 0 0 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 3 3 3 3 0 0 0 3 3 0 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 0 3 3 3
 0 0 3 3 0 1 1 1 1 1 1 1 1 1 3 3 3 3 1 1 1 3 1 1 1 3 1 1 1 1 1 1 1 0 1 3 1
 1 1 1 1 3 1 1 3 3 1 1 1 3 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1]


In [156]:
assignedCluster = np.array([assignedCluster])
print(assignedCluster)


[[4 2 2 2 4 4 4 4 4 4 4 4 0 0 0 2 0 0 4 4 0 0 0 4 4 4 2 2 2 2 2 2 2 2 3 4 0
  3 0 3 3 1 4 0 0 0 1 1 1 0 4 0 4 3 3 0 0 0 3 3 1 1 1 1 3 1 1 3 0 0 0 3 0 1
  1 3 3 1 3 0 3 3 0 0 3 3 0 0 0 3 3 0 0 0 0 3 3 3 3 1 0 0 1 1 1 0 0 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 3 3 3 3 0 0 0 3 3 0 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 0 3 3 3
  0 0 3 3 0 1 1 1 1 1 1 1 1 1 3 3 3 3 1 1 1 3 1 1 1 3 1 1 1 1 1 1 1 0 1 3 1
  1 1 1 1 3 1 1 3 3 1 1 1 3 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1]]


In [157]:
new=np.concatenate((allFactors[:,:1], assignedCluster.T), axis=1)
print (new)

[[ 656.3    4. ]
 [ 763.     2. ]
 [ 884.3    2. ]
 [ 884.3    2. ]
 [ 711.5    4. ]
 [ 534.2    4. ]
 [ 621.     4. ]
 [ 532.6    4. ]
 [ 530.6    4. ]
 [ 721.1    4. ]
 [ 709.5    4. ]
 [ 711.5    4. ]
 [ 348.2    0. ]
 [ 320.1    0. ]
 [ 381.     0. ]
 [ 784.3    2. ]
 [ 383.9    0. ]
 [ 435.3    0. ]
 [ 554.9    4. ]
 [ 707.7    4. ]
 [ 438.4    0. ]
 [ 388.3    0. ]
 [ 377.3    0. ]
 [ 728.8    4. ]
 [ 727.5    4. ]
 [ 727.5    4. ]
 [ 884.3    2. ]
 [ 884.3    2. ]
 [ 884.3    2. ]
 [ 884.3    2. ]
 [ 884.3    2. ]
 [ 884.3    2. ]
 [ 884.3    2. ]
 [ 884.3    2. ]
 [ 262.5    3. ]
 [ 572.4    4. ]
 [ 453.6    0. ]
 [ 303.1    3. ]
 [ 415.7    0. ]
 [ 303.9    3. ]
 [ 216.8    3. ]
 [  59.7    1. ]
 [ 717.6    4. ]
 [ 332.6    0. ]
 [ 457.4    0. ]
 [ 326.8    0. ]
 [ 147.5    1. ]
 [  66.7    1. ]
 [ 112.3    1. ]
 [ 496.3    0. ]
 [ 552.4    4. ]
 [ 359.5    0. ]
 [ 586.2    4. ]
 [ 223.5    3. ]
 [ 231.9    3. ]
 [ 386.     0. ]
 [ 459.6    0. ]
 [ 343.7    0. ]
 [ 259.6    3.

I princip kan det här göras med många fler näringsvärden där varje näringsvärde blir en "dimension". 

Eftersom vi vill visa klustringen i ett 3D-diagram har vi begränsat oss till tre dimensioner i exemplet.

## Visualisering med Plotly

I Plotly svarar varje näringsvärde mot en axel.

För att Plotly ska kunna plotta varje axel för sig transponerar vi data så vi får en rad för x-axeln, en för y-axeln och en för z.

Ett exempel:

<pre>
[[   0.     0.     0.     0.     0.5]
 [  71.    85.   100.   100.    80. ]
 [   7.     2.8    0.     0.     0.5]]
</pre>

In [158]:
dataToChart = dataSet.transpose()

In [159]:
if debug:
    print (dataToChart)

[[  2.74600000e+03   3.19260000e+03   3.70000000e+03 ...,   1.09100000e+02
    1.23400000e+02   1.31300000e+02]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   4.40000000e+00
    4.10000000e+00   4.70000000e+00]
 [  7.10000000e+01   8.50000000e+01   1.00000000e+02 ...,   1.00000000e-01
    1.50000000e-01   3.00000000e-01]
 [  7.00000000e+00   2.80000000e+00   0.00000000e+00 ...,   1.00000000e+00
    1.56000000e+00   1.13000000e+00]]


För att visualisera vilka kluster som ett livsmedel ligger så ger vi varje kluster olika färger. I Plotly svarar detta mot att varje kluster hamnar i ett separat "trace"

Därför behöver vi gå igenom vårt *dataToChart* och separera ut vilka livsmedel som hör till vilket kluster.

Då behöver vi en tvådimensionell array där storleken på första dimensionen är antal kluster och storleken på andra dimensionen är tre (eftersom vi visualiserar tre dimensioner)

In [160]:
# create multi dimensional array of data by label
segmentedData = [[[] for _ in range(4)] for _ in range(numClusters)]


Det är lite krångligt att visualisera den här strukturen. Så här blir den med fem kluster (numClusters) och tre faktorer för klustringen (fett, protein, kolhydrater) 
<pre>
[
    [[], [], []], 
    [[], [], []], 
    [[], [], []], 
    [[], [], []], 
    [[], [], []]
]
</pre>

(Om man väljer att klustra med fler än tre faktorer är det svårt att visualisera alla, utan man måste välja bort någon faktor)

In [161]:
jobbigArray = np.array(segmentedData)
print (jobbigArray)

[]


In [162]:
for num, cluster in enumerate(assignedCluster):
        if debug:
            print (str(num) + ' ' + str(cluster))
        segmentedData[cluster][0].append(dataToChart[0][num])
        segmentedData[cluster][1].append(dataToChart[1][num])
        segmentedData[cluster][2].append(dataToChart[2][num])
        segmentedData[cluster][3].append(dataToChart[3][num])
        
jobbigArray = np.array(segmentedData)
print (jobbigArray.ndim)

0 [4 2 2 2 4 4 4 4 4 4 4 4 0 0 0 2 0 0 4 4 0 0 0 4 4 4 2 2 2 2 2 2 2 2 3 4 0
 3 0 3 3 1 4 0 0 0 1 1 1 0 4 0 4 3 3 0 0 0 3 3 1 1 1 1 3 1 1 3 0 0 0 3 0 1
 1 3 3 1 3 0 3 3 0 0 3 3 0 0 0 3 3 0 0 0 0 3 3 3 3 1 0 0 1 1 1 0 0 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 3 3 3 3 0 0 0 3 3 0 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 0 3 3 3
 0 0 3 3 0 1 1 1 1 1 1 1 1 1 3 3 3 3 1 1 1 3 1 1 1 3 1 1 1 1 1 1 1 0 1 3 1
 1 1 1 1 3 1 1 3 3 1 1 1 3 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1]


TypeError: only integer scalar arrays can be converted to a scalar index

In [163]:
if debug:
    print(segmentedData)

[[[], [], [], []], [[], [], [], []], [[], [], [], []], [[], [], [], []], [[], [], [], []]]


In [164]:
# create traces for plotly
traces = []
baseColor = 100
i = 0
while i < numClusters:
    trace = go.Scatter3d(
        x=segmentedData[i][0],
        y=segmentedData[i][1],
        z=segmentedData[i][3],
        mode='markers',
        marker=dict(
            size=12,
            line=dict(
                color='rgba(baseColor+(i*2), baseColor+(i*2), baseColor+(i*2), 0.14)',
                width=0.5
            ),
            opacity=0.8
        ),
        # @todo: fix names list for plotly
        #text=names
    )
    traces.append(trace)
    i+=1


In [165]:
layout = go.Layout(
        scene=go.Scene(
            xaxis=go.XAxis(title='Carbs', tickprefix='Carbs ', showtickprefix='first'),
            yaxis=go.YAxis(title='Fat', tickprefix='Fat ', showtickprefix='first'),
            zaxis=go.ZAxis(title='Protein', tickprefix='Protein ', showtickprefix='first')
        ),
        height = 800,
        width = 800,
    )

In [166]:
fig = go.Figure(data=traces, layout=layout)    

In [167]:
py.iplot(fig, filename='table1')

In [168]:
conn = sqlite3.connect('livs.db')  # Create db and establish connection
conn.row_factory = sqlite3.Row
curs = conn.cursor()
columnObject = []

        #Kolla om klusterkolumnen redan finns, annars lägg till den        
for row in conn.execute('PRAGMA table_info (livs);'):
    columnObject.append(row)
columns = np.array(columnObject)
newColumn = 'Kluster'
addColumn = True
for row in columns:
    print (row)
    if newColumn == row[1]:
        addColumn = False
        print (addColumn, newColumn)
if addColumn:
    conn.execute('ALTER TABLE livs ADD COLUMN Kluster REAL;')

#
conn.close()

[0 'Livsmedelsnamn' 'TEXT' 0 None 0]
[1 'Livsmedelsnummer' 'TEXT' 0 None 0]
[2 'Energi_kcal' 'REAL' 0 None 0]
[3 'Energi_kJ' 'REAL' 0 None 0]
[4 'Kolhydrater_g' 'REAL' 0 None 0]
[5 'Fett_g' 'REAL' 0 None 0]
[6 'Protein_g' 'REAL' 0 None 0]
[7 'Fibrer_g' 'REAL' 0 None 0]
[8 'Vatten_g' 'REAL' 0 None 0]
[9 'Alkohol_g' 'REAL' 0 None 0]
[10 'Aska_g' 'REAL' 0 None 0]
[11 'Monosackarider_g' 'REAL' 0 None 0]
[12 'Disackarider_g' 'REAL' 0 None 0]
[13 'Sackaros_g' 'REAL' 0 None 0]
[14 'Fullkorn_totalt_g' 'REAL' 0 None 0]
[15 'Sockerarter_g' 'REAL' 0 None 0]
[16 'Summa_mattade_fettsyror_g' 'REAL' 0 None 0]
[17 'Fettsyra_4_0_10_0_g' 'REAL' 0 None 0]
[18 'Fettsyra_12_0_g' 'REAL' 0 None 0]
[19 'Fettsyra_14_0_g' 'REAL' 0 None 0]
[20 'Fettsyra_16_0_g' 'REAL' 0 None 0]
[21 'Fettsyra_18_0_g' 'REAL' 0 None 0]
[22 'Fettsyra_20_0_g' 'REAL' 0 None 0]
[23 'Summa_enkelomattade_fettsyror_g' 'REAL' 0 None 0]
[24 'Fettsyra_16_1_g' 'REAL' 0 None 0]
[25 'Fettsyra_18_1_g' 'REAL' 0 None 0]
[26 'Summa_fleromattade_fet

## Lägga in kluster i databasen


In [169]:
print(new)

for row in new:
    print (row[0])



[[ 656.3    4. ]
 [ 763.     2. ]
 [ 884.3    2. ]
 [ 884.3    2. ]
 [ 711.5    4. ]
 [ 534.2    4. ]
 [ 621.     4. ]
 [ 532.6    4. ]
 [ 530.6    4. ]
 [ 721.1    4. ]
 [ 709.5    4. ]
 [ 711.5    4. ]
 [ 348.2    0. ]
 [ 320.1    0. ]
 [ 381.     0. ]
 [ 784.3    2. ]
 [ 383.9    0. ]
 [ 435.3    0. ]
 [ 554.9    4. ]
 [ 707.7    4. ]
 [ 438.4    0. ]
 [ 388.3    0. ]
 [ 377.3    0. ]
 [ 728.8    4. ]
 [ 727.5    4. ]
 [ 727.5    4. ]
 [ 884.3    2. ]
 [ 884.3    2. ]
 [ 884.3    2. ]
 [ 884.3    2. ]
 [ 884.3    2. ]
 [ 884.3    2. ]
 [ 884.3    2. ]
 [ 884.3    2. ]
 [ 262.5    3. ]
 [ 572.4    4. ]
 [ 453.6    0. ]
 [ 303.1    3. ]
 [ 415.7    0. ]
 [ 303.9    3. ]
 [ 216.8    3. ]
 [  59.7    1. ]
 [ 717.6    4. ]
 [ 332.6    0. ]
 [ 457.4    0. ]
 [ 326.8    0. ]
 [ 147.5    1. ]
 [  66.7    1. ]
 [ 112.3    1. ]
 [ 496.3    0. ]
 [ 552.4    4. ]
 [ 359.5    0. ]
 [ 586.2    4. ]
 [ 223.5    3. ]
 [ 231.9    3. ]
 [ 386.     0. ]
 [ 459.6    0. ]
 [ 343.7    0. ]
 [ 259.6    3.

Dags att läsa ner resultatet av klustringen till databasen

In [170]:
conn = sqlite3.connect('livs.db')  # Create db and establish connection
conn.row_factory = sqlite3.Row
curs = conn.cursor()


for row in new:
    print(row[1], row[0])
    sql = 'UPDATE livs SET Kluster = '+row[1]+' WHERE Livsmedelsnummer = "'+row[0]+'";'
    print(sql)
    curs.execute(sql)
#curs.execute('UPDATE livs SET Kluster = 1 WHERE Livsmedelsnummer = "2";')
conn.commit()
conn.close()

4.0 656.3


TypeError: ufunc 'add' did not contain a loop with signature matching types dtype('<U32') dtype('<U32') dtype('<U32')

In [171]:
conn = sqlite3.connect('livs.db')  # Create db and establish connection
conn.row_factory = sqlite3.Row
curs = conn.cursor()
   
KlusterObject = []
for row in curs.execute('SELECT Livsmedelsnamn from livs WHERE Kluster = 0;'):
    #print(row)
    KlusterObject.append(row)

        
Kluster = np.array(KlusterObject)
print (Kluster)

conn.close()

[['Dressing majonnäs lätt fett ca 15%']
 ['Dressing fett 0%']
 ['Mimosasallad']
 ['Rödbetssallad gräddfil']
 ['Skaldjurssallad m majonnäsdressing']
 ['Bearnaisesås fryst el pulver tillagad']
 ['Hollandaisesås fryst el pulver tillagad']
 ['Keso färskost fett 4%']
 ['Keso färskost ananas apelsin fett 3%']
 ['Keso färskost chili pesto lök feta fett 3,5-5%']
 ['Färskost på filmjölk fett ca 8%']
 ['Färskost på filmjölk o gräddfil fett ca 14%']
 ['Kvarg färskost fett 1% Kesella lätt']
 ['Kvarg färskost fett 10% Kesella']
 ['Mjukost smältost fett 4%']
 ['Färskost lätt fett 15% typ Philadelphia']
 ['Färskost extra light fett 5% typ Philadelphia']
 ['Bröstmjölk humanmjölk fett 4,6%']
 ['Filmjölk fett 3%']
 ['Kondenserad mjölk konserv fett ca 8%']
 ['Långfil fett 3%']
 ['Lättfil fett 0,5%']
 ['Milkshake choklad jordgubb']
 ['Mjölk fett 3%']
 ['Yoghurt naturell fett 3 %']
 ['Fruktyoghurt fett 2,5%']
 ['Fruktyoghurt lätt fett 0,5%']
 ['Mjölk fett 4,2 % typ lantmjölk']
 ['A-fil  fett 3%']
 ['Mellan